Mini Portfolio Data Processing

# No. 1

1. Filter kolom `MonthlyRevenue` dan `TotalRecurringCharge` sehingga tidak ada yang null dan tidak ada yang bernilai 0.
2. Buatlah kolom baru dinamakan `revenue per recurring charge`, merupakan pembagian `MonthlyRevenue / TotalRecurringCharge`
3. Hapus baris dimana kolom baru tersebut > 9
4. Buat lagi kolom baru dinamakan `DailyRevenue`, yang merupakan `MonthlyRevenue /30`

Setelah ke-4 steps dilakukan, berapa nilai tengah dari kolom DailyRevenue?


In [2]:
#import libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
#read dataset
df = pd.read_csv('01 telecom_revenue.csv')

df_filter1 = df.loc[(df['MonthlyRevenue'].notnull())
                    & (df['TotalRecurringCharge'].notnull())]
df_filter2 = df_filter1.loc[(df_filter1['MonthlyRevenue'] != 0)
                            & (df_filter1['TotalRecurringCharge'] != 0)]

df_filter2['per_recurring_charge'] = df_filter2['MonthlyRevenue'] / df_filter2['TotalRecurringCharge']
df_drop = df_filter2.loc[~(df_filter2['per_recurring_charge'] > 9)]
df_drop['DailyRevenue'] = df_drop['MonthlyRevenue'] / 30
df_drop.describe()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,OverageMinutes,RoamingCalls,per_recurring_charge,DailyRevenue
count,9.950000e+03,9950.000000,9950.000000,9950.000000,9950.000000,9950.000000,9950.000000,9950.000000
mean,3.202302e+06,59.376125,531.805729,47.179196,40.749749,1.272211,1.311866,1.979204
std,1.165365e+05,44.606196,529.810131,24.008914,97.164859,9.127499,0.809471,1.486873
min,3.000022e+06,-5.860000,0.000000,-5.000000,0.000000,0.000000,-10.970000,-0.195333
25%,3.101746e+06,34.065000,165.000000,30.000000,0.000000,0.000000,0.913514,1.135500
50%,3.201780e+06,48.675000,371.000000,45.000000,3.000000,0.000000,1.051208,1.622500
75%,3.305346e+06,71.992500,733.000000,60.000000,42.000000,0.300000,1.388958,2.399750
max,3.399946e+06,855.500000,7359.000000,400.000000,2278.000000,692.400000,8.962727,28.516667


In [4]:
print('Nilai tengah atau nilai median dari Daily Revenue bernilai', df_drop['DailyRevenue'].median())

Nilai tengah atau nilai median dari Daily Revenue bernilai 1.6225


# No. 2

## Section A

Ada berapa customerID yang memiliki MonthlyRevenue > 50, tetapi CustomerCareCalls is NULL?

In [10]:
#read dataset
df1 = pd.read_csv('01 telecom_revenue.csv')
df2 = pd.read_csv('02 telecom_usage.csv')

df_merge = df1.merge(df2,
                     on = 'CustomerID',
                     how = 'left')
df_filter = df_merge.loc[(df_merge['MonthlyRevenue'] > 50)
                         & (df_merge['CustomerCareCalls'].isnull())]
print('Total Customer yang memiliki MonthlyRevenue > 50, tetapi CustomerCareCalls is NULL sejumlah',df_filter['CustomerID'].nunique())

Total Customer yang memiliki MonthlyRevenue > 50, tetapi CustomerCareCalls is NULL sejumlah 4303


## Section B

Jika dibandingkan per Occupation, pada Occupation apa yang MonthlyRevenue paling tinggi dan paling rendah secara rata-rata dari seluruh customerID pada Section (a) di atas?

In [9]:
df_filterOccupation = df_filter.groupby(['Occupation']).agg({'MonthlyRevenue' : 'mean'})
print('Tertinggi = ' ,  df_filterOccupation.sort_values(['MonthlyRevenue'], ascending = False).reset_index()['Occupation'][0])
print('Terendah = ' ,df_filterOccupation.sort_values(['MonthlyRevenue'], ascending = True).reset_index()['Occupation'][0])

Tertinggi =  Crafts
Terendah =  Retired


# No. 3

## Section A

Bandingkan churn rate pada semua Occupation!

In [12]:
import pandas as pd
data = pd.read_csv('01 telecom_revenue.csv')
data_filter= data.groupby(['Occupation', 'Churn']).agg({'CustomerID' : pd.Series.nunique}).reset_index()
data_filter['Total'] = data_filter.groupby(['Occupation'])['CustomerID'].transform('sum')
data_filter['ChurnRate'] = data_filter['CustomerID'] / data_filter['Total']
data_a = data_filter.loc[data_filter['Churn'] == 'Yes'].reset_index(drop = True)
data_a.sort_values(['ChurnRate'], ascending = False)

,Occupation,Churn,CustomerID,Total,ChurnRate
7,Student,Yes,25,69,0.362319
0,Clerical,Yes,55,176,0.312500
6,Self,Yes,50,170,0.294118
3,Other,Yes,2170,7409,0.292887
1,Crafts,Yes,84,305,0.275410
4,Professional,Yes,455,1690,0.269231
2,Homemaker,Yes,8,33,0.242424
5,Retired,Yes,33,148,0.222973


## Section B

Pada Occupation dengan churn rate paling kecil dan paling besar,Berapa rata-rata MontlyRevenue pada kedua Occupation tersebut? <br>
Dan berapa ABSOLUTE(selisih) dari kedua rata-ratanya ?

In [33]:
occMax = data_a.sort_values(['ChurnRate'], ascending = False).reset_index()['Occupation'][0]
occMin = data_a.sort_values(['ChurnRate'], ascending = True).reset_index()['Occupation'][0]
occMax, occMin

('Student', 'Retired')

In [61]:
import pandas as pd
df = pd.read_csv('01 telecom_revenue.csv')
df_filter = df.groupby(['Occupation']).agg({'MonthlyRevenue' : 'mean'}).reset_index()
Max = df_filter.loc[df_filter['Occupation'] == occMax]['MonthlyRevenue'].values
Min = df_filter.loc[df_filter['Occupation'] == occMin]['MonthlyRevenue'].values
print('Mean occupation churn rate terbesar = ' , Max )
print('Mean occupation churn rate terbesar = ' , Min )
print('Absolute(selisih) = ' , Max - Min)

Mean occupation churn rate terbesar =  [63.24275362]
Mean occupation churn rate terbesar =  [43.72966216]
Absolute(selisih) =  [19.51309146]


# No. 4

Buatlah 3 dataframe mengikuti petunjuk berikut:

df_1 :  filter dataset 01 telecom_revenue.csv dengan kondisi `MonthlyRevenue > 10` & `Occupation in (Professional, Student, and Crafts)` kemudian lakukan reset_index(drop=True) pada df_1

df_2 :  filter dataset 01 telecom_revenue.csv dengan kondisi `CustomerCareCalls > 10` dan `MonthlyRevenue kurang dari atau sama dengan XXX`. Dimana XXX adalah rata-rata MonthlyRevenue saat Churn=No pada output df_1. Kemudian lakukan reset_index(drop=True) pada df_2

df_3 :  filter dataset 01 telecom_revenue.csv pada index berikut: [2331, 3165, 3867, 4030, 4570] kemudian lakukan reset_index(drop=True) pada df_3 <br>Gabungkan df_1 , df_2 dan df_3 secara union.<br>
<br>
Pada Occupation apa, Churn rate >= 0.4 ?


In [95]:
import pandas as pd
df_1 = pd.read_csv('01 telecom_revenue.csv')
df_1 = df_1.loc[(df_1['MonthlyRevenue']>10)
                & (df_1['Occupation'].isin(['Professional', 'Student', 'Crafts']))].reset_index(drop = True)
df_1

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,OverageMinutes,RoamingCalls,Occupation
0,3397170,No,83.51,290.0,93.0,0.0,0.0,Professional
1,3274822,No,30.83,273.0,40.0,0.0,0.8,Professional
2,3394258,No,52.72,1501.0,62.0,0.0,0.0,Crafts
3,3056178,Yes,54.09,412.0,40.0,46.0,0.0,Professional
4,3065718,No,43.06,110.0,35.0,0.0,3.3,Crafts
...,...,...,...,...,...,...,...,...
2018,3015822,No,46.10,203.0,38.0,30.0,0.0,Crafts
2019,3323878,No,44.26,419.0,45.0,2.0,0.0,Professional
2020,3050290,Yes,51.06,192.0,30.0,74.0,0.0,Professional
2021,3119258,No,39.49,352.0,45.0,0.0,1.7,Crafts


In [104]:
rata_rata = df_1.loc[df_1['Churn'] == 'No']['MonthlyRevenue'].mean()
rata_rata
df_2 = pd.read_csv('01 telecom_revenue.csv').merge(pd.read_csv('02 telecom_usage.csv'), on = 'CustomerID', how = 'outer')
df_2 = df_2.loc[(df_2['CustomerCareCalls'] > 10)
               & (df_2['MonthlyRevenue'] <= rata_rata)].reset_index(drop = True)
df_2

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,OverageMinutes,RoamingCalls,Occupation,DroppedCalls,BlockedCalls,UnansweredCalls,CustomerCareCalls
0,3344578,No,45.63,731.0,40.0,14.0,0.0,Professional,13.0,2.3,35.7,10.7
1,3178106,No,43.26,362.0,30.0,36.0,0.0,Professional,3.3,0.0,13.3,15.3
2,3345794,No,34.99,735.0,45.0,0.0,0.0,Professional,12.3,1.7,56.0,10.3
3,3360210,No,35.56,316.0,30.0,2.0,0.0,Other,5.0,2.7,104.0,18.0
4,3273278,No,37.57,782.0,48.0,0.0,0.0,Other,0.0,1.3,23.0,15.7
5,3022506,Yes,56.63,286.0,60.0,0.0,3.4,Other,10.7,0.3,1.0,12.7
6,3300078,No,50.56,1482.0,60.0,0.0,0.0,Other,26.7,1.0,93.7,19.7
7,3191166,No,42.83,870.0,55.0,9.0,6.3,Other,4.0,0.0,2.3,32.0
8,3115706,No,33.48,346.0,30.0,39.0,0.0,Professional,5.3,3.0,32.3,17.7
9,3228526,No,32.44,102.0,40.0,0.0,2.0,Crafts,3.0,1.0,11.7,11.0


In [107]:
df_3 = pd.read_csv('01 telecom_revenue.csv').iloc[[2331, 3165, 3867, 4030, 4570]].reset_index(drop = True)
final = df_1.append([df_2, df_3])
final = final.groupby(['Occupation', 'Churn']).agg({'CustomerID' : pd.Series.nunique}).reset_index()
final['Total'] = final.groupby(['Occupation'])['CustomerID'].transform('sum')
final['ChurnRate'] = final['CustomerID'] / final['Total']
final_clean = final.loc[final['Churn'] == 'Yes']
final_clean.loc[final_clean['ChurnRate'] >= 0.4]

,Occupation,Churn,CustomerID,Total,ChurnRate
0,Clerical,Yes,1,1,1.000000
4,Other,Yes,4,9,0.444444


Ada 2 Occupation yang lebih dari sama dengan 0.4 yaitu `Clerial` dan `Other`